In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd
import json

# Ignore all warnings
warnings.filterwarnings("ignore")

# Specify the path to your existing CSV file
csv_file_path = 'amazon_pricing1.csv'

# Read ASINs from 'ASINs_validation.csv'
asins_file_path = 'ASINs_validation.csv'
with open(asins_file_path, 'r') as asins_file:
    # Assuming ASINs are in the first column of the CSV file
    asins_list = [row[0] for row in csv.reader(asins_file) if row]

# Create a list to store failed ASINs
failed_asins_list = []

# Open the CSV file in append mode
with open(csv_file_path, 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)

    # Check if the file is empty and write headers if needed
    if f.tell() == 0:
        writer.writerow(['ASIN','Title', 'Price'])

    # Iterate through each ASIN and fetch data simultaneously
    for asin in asins_list:
        retry_count = 2  # Number of retries
        while retry_count > 0:
            # Create the URL for the current ASIN
            base_url = "https://www.amazon.com/dp/"
            url = base_url + asin
            print(f"Processing ASIN: {asin}")
            
            # Make a request to the Amazon product page
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept-Language": "en-US,en;q=0.9",
                "Accept-Ch": "ect,rtt,downlink,device-memory,sec-ch-device-memory,viewport-width,sec-ch-viewport-width,dpr,sec-ch-dpr,sec-ch-ua-platform-version",
                "Accept-Ch-Lifetime": "86400",
                "Alt-Svc": "h3=\":443\"; ma=86400",
                "Accept-Encoding": "gzip, deflate, br",
                "authority": "www.amazon.com",
                "method": "POST",
                "path": "/vap/ew/componentbuilder",
                "scheme": "https",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "DNT": "1",
                "Connection": "close",
                "Upgrade-Insecure-Requests": "1",
                "Cookie": """ubid-main=130-3904539-0911866; sid="YBT77Xu/Zn8g/PpYuT6V3w==|lBS9rSAADjIMjOMeiPK6+eaAjEpeL/AluT86houlaKo="; session-id-eu=257-9075817-4124757; ubid-acbuk=257-4076612-4476044; session-id-time-eu=2338210524l; regStatus=pre-register; AMCV_7742037254C95E840A4C98A6%40AdobeOrg=1585540135%7CMCIDTS%7C19766%7CMCMID%7C91852539536399208512041036142468214903%7CMCAAMLH-1708320263%7C7%7CMCAAMB-1708320263%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1707722664s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.0; aws-target-data=%7B%22support%22%3A%221%22%7D; aws-target-visitor-id=1707715466468-154002.44_0; i18n-prefs=USD; lc-main=en_US; s_pers=%20s_fid%3D3FA5DB706AF56DBB-01AFBADBF6EA79F3%7C1880690221255%3B%20s_dl%3D1%7C1722925621256%3B%20gpv_page%3DV3%2520BRND%2520US%2520Gateway%7C1722925621257%3B%20s_ev15%3D%255B%255B%2527NSGoogle%2527%252C%25271722923821260%2527%255D%255D%7C1880690221260%3B; session-id=138-4336706-3226901; x-amz-captcha-1=1724904507875364; x-amz-captcha-2=957Jncm/4Hwx3OkbtI6wAg==; s_nr=1729145897587-Repeat; s_vnum=2152521773541%26vn%3D4; s_dslv=1729145897588; sst-main=Sst1|PQEpsGi4R5yQMwfmmJNqGbLbCZ5ygUh10viQ7AQBrFdIpDdCpsqUzFJGuvfXF2WbKrXXw5OIpFMmOqFWGZbgpHHTQl1oxxlsaYsOPER9H0Wi1HuU4c0SwDNFQ-gS4_J2DRgBMpmZ07LpIxEtcOVlg8xlCvq-Uq61XibeJ5cIMVDHZ6IvWpAbE5FUhP-R6sOhOhVw-FXps5bcdqN_nhYoM3x7NdXNu0Whx_cJ55rvOu2jBpA4zTZB4Yw67feoWfQJofJ2Kn5TTksR3rqtXmPAC_VmxuedsRDWtGvZ10etUPBfgDQ; at-main=Atza|IwEBIBjJu2Px1roLXhiR4ijHmqZEEblycWC0n4VanLGXrFAAYdmQGMRBl3BRgdftYQBBA0Rp2lvvEFk1g8MCm29nkkpeXvZwjJnxjTRvnbHnJYubYIZeL5CB0bolprLW1BlAe08RqtgfybdoRo_mZ8OxRCfzNa7WD_xgMtqnhl4ExxWodpoEWape-8Gb03RuE9J3vipg6Q2wiDSQDM0Mt1eaVAf1-thZYMLmj1hhBjmERdrB36hCGuVSSv_HHqBmB1bSiUP-JcIEHHaggaMG-qI6Prlx8-kLj1wYVYZ2_luT3wnv9g; sess-at-main="Xu7jbTDl4sptWCI4g/pzElB3Dp3EGpi/2o2QJiJMIyY="; session-id-time=2082787201l; x-main=gEtXm0ar@kcOKvon@HY0NDUOMWInxwZ?uD?egNQ9hXEYQ?xfhcZppFr5ZBirG8SN; ph_phc_tGCcl9SINhy3N0zy98fdlWrc1ppQ67KJ8pZMzVZOECH_posthog=%7B%22distinct_id%22%3A%22amzn1.account.ocid.A3E4QLZBKGG966%22%2C%22%24sesid%22%3A%5B1732260722422%2C%22019352c8-c9d0-7d07-adda-7c66c6c188d4%22%2C1732260710863%5D%2C%22%24epp%22%3Atrue%7D; session-token=tNTjVClRYWKVd659KAKkWsJR0D3m0m7PE5DftQlVdFowkVXHm9mUwseN4yO64J5HrblqhbrSA7+/kpFSY/ubmOChU1xlcIX94L33+ZhkbwqHLvA96CTclhUvr3qklJ98t7H4rVRHOf+NSvaNwOjbxOiIPdi+iIb5DJWSsQ8Ls+7tMbBa4ZfXu3JT+2g+byzhRZhP0cYX/SSIRXvhby0B/5XAPcvIT9fW8WHNAt/agZVsGm/Y9OMl//9zPUbm/7dfwfas7g6f1LfFFpNEfQSmYKTJplbqIVnAmAV02pCo7dfONjS37Oq7LNFQ73u+fPnQhP/Kiesj2Qy4ANxRTtnf0xJLFTaC217Vql7NwTeRcHMchiziO7yGr10akZkhxEo9; csm-hit=tb:s-P2EVSD6MY5DB8EBQSDA9|1732261043186&t:1732261045182&adb:adblk_no"""
            }
            try:
                page = requests.get(url, headers=headers)
                time.sleep(0.8)  # Introduce a delay between requests
            except Exception as e:
                print(f"Error fetching data for ASIN {asin}: {e}")
                # Append the ASIN to the failed_asins_list
                failed_asins_list.append({'ASIN': asin, 'Reason': str(e)})
                continue  # Retry if there is an error

            # Parse the HTML content
            soup1 = BeautifulSoup(page.content, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            # Extract data from the page
            title = soup2.find(id='productTitle').get_text().strip() if soup2.find(id='productTitle') else 'N/A'
            print(f"Title: {title}")
            
#             # Find the Best Sellers Rank row by looking for the <th> containing "Best Sellers Rank"
#             bsr_row = soup2.find('div', id='prodDetails')            

#             # Extract the corresponding <td> if the Best Sellers Rank section is found
#             if bsr_row:
#                 bsr_data = bsr_row.find_next('Best Sellers Rank').get_text(separator=" ", strip=True)
#                 print(f"Best Sellers Rank: {bsr_data}")
#             else:
#                 bsr_data = 'N/A'
#                 print("Best Sellers Rank not found.")

                        # Check if the item is unavailable based on text
                        # Check if the item is unavailable based on text
            price = 'N/A'
            unavailable_count = soup2.get_text().count("Currently unavailable")            

            # Check if the item is unavailable
            if unavailable_count >= 2:
                price = 'N/A'
                print(f"ASIN {asin} is unavailable. Setting price to 'N/A'.")
            else:
                # Attempt to find price in various sections
                if price == 'N/A':
                    print("Attempting to find price in multiple sections.")
                    
                    # List of potential sections with price spans
                    potential_ids = [
                        'corePriceDisplay_desktop_feature_div',  # Main target
                        'priceblock_ourprice',                  # Fallback for list price
                        'priceblock_dealprice',                 # For deals
                        'corePrice_feature_div',                # Another possible section
                    ]
                    
                    price_section = None
                    for section_id in potential_ids:
                        price_section = soup2.find('div', id=section_id)
                        if price_section:
                            break
                    
                    if not price_section:
                        # Check for price in table structure with 'Bundle Price' and 'Bundle List Price'
                        table_section = soup2.find('table', class_='a-lineitem a-align-top')
                        if table_section:
                            # Extract 'Bundle List Price'
                            bundle_list_price_span = table_section.find('span', class_='a-price a-text-price a-size-base')
                            if bundle_list_price_span:
                                bundle_list_price = bundle_list_price_span.find('span', class_='a-offscreen')
                                if bundle_list_price:
                                    price = bundle_list_price.get_text().strip()
                                    print(f"Bundle List Price found: {price}")
                            
                            # Extract 'Bundle Price'
                            if price == 'N/A':  # Only search if Bundle List Price is not found
                                bundle_price_span = table_section.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay')
                                if bundle_price_span:
                                    bundle_price = bundle_price_span.find('span', class_='a-offscreen')
                                    if bundle_price:
                                        price = bundle_price.get_text().strip()
                                        print(f"Bundle Price found: {price}")
                    
                    # Broader search for any price element (fallback)
                    if price == 'N/A':
                        print("Attempting to find price in any span with 'a-price' class.")
                        price_spans = soup2.find_all('span', class_='a-price')
                        for price_span in price_spans:
                            offscreen_price = price_span.find('span', class_='a-offscreen')
                            if offscreen_price and offscreen_price.get_text().strip():
                                price = offscreen_price.get_text().strip()
                                print(f"Fallback price found: {price}")
                                break  # Exit loop once a valid price is found            

                    if price_section:
                        # Search for the price in the section
                        price_block = price_section.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
                        if price_block:
                            price_symbol = price_block.find('span', class_='a-price-symbol')
                            price_whole = price_block.find('span', class_='a-price-whole')
                            price_fraction = price_block.find('span', class_='a-price-fraction')            

                            if price_symbol and price_whole and price_fraction:
                                price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
                                print(f"Price found: {price}")
                            else:
                                print("Price components not found in the specified structure.")
                        else:
                            print("Price block not found in the identified section.")
                    else:
                        print("No relevant price section found.")


                
                
#                 # Attempt 1: Check for price within the main price block
#                 if price == 'N/A':
#                     print("Attempting to find price in the specified price structure.")
#                     price_block = soup2.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
#                     if price_block:
#                         # Extract the price components
#                         price_symbol = price_block.find('span', class_='a-price-symbol')
#                         price_whole = price_block.find('span', class_='a-price-whole')
#                         price_fraction = price_block.find('span', class_='a-price-fraction')
                        
#                         if price_symbol and price_whole and price_fraction:
#                             price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
#                             print(f"Price found: {price}")
#                         else:
#                             print("Price components not found in the specified structure.")
#                     else:
#                         print("Price block with specified class not found.")


#                 # Attempt 2: Check for JSON data containing price information
#                 price_data_div = soup2.find('div', class_='a-section aok-hidden twister-plus-buying-options-price-data')
#                 if price_data_div:
#                     print("Found JSON data for price.")
#                     price_data_text = price_data_div.get_text()
#                     try:
#                         price_data = json.loads(price_data_text)
#                         price = price_data['desktop_buybox_group_1'][0]['displayPrice']
#                         # print(f"Price found from JSON data: {price}")
#                     except (KeyError, IndexError, json.JSONDecodeError) as e:
#                         print(f"Error parsing JSON data: {e}")

#                 # Attempt 3: Check for nested price elements
#                 if price == 'N/A':
#                     # print("Attempting to find price in nested price elements.")
#                     price_container = soup2.find('span', class_='a-price')
#                     if price_container:
#                         price_whole = price_container.find('span', class_='a-price-whole')
#                         price_fraction = price_container.find('span', class_='a-price-fraction')
#                         if price_whole and price_fraction:
#                             price = f"${price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
#                             # print(f"Price found in nested elements: {price}")

#                 # Attempt 4: Handle cases with complex structures or different classes
#                 if price == 'N/A':
#                     # print("Attempting to find price in alternative structures.")
#                     alternative_price = soup2.find('span', class_='a-size-large a-color-price savingPriceOverride')
#                     if alternative_price:
#                         price = alternative_price.get_text().strip()
#                         # print(f"Price found in alternative structure: {price}")
            
#             # Debugging: Output the HTML content
#             with open("page_content.html", "w", encoding="utf-8") as file:
#                 file.write(soup2.prettify())
#             print("Saved page content to 'page_content.html' for inspection.")



            # Check if title is 'N/A' and retry if needed
            if title == 'N/A':
                retry_count -= 1
                # print(f"Retrying for ASIN {asin}. Retries left: {retry_count}")
            else:
                break  # Break out of the retry loop if successful

        # If all retries failed, print an error message
        if retry_count == 0:
            # print(f"Failed to fetch data for ASIN {asin} after multiple retries.")
            # Append the ASIN to the failed_asins_list
            failed_asins_list.append({'ASIN': asin, 'Reason': 'Max retries reached'})
            continue

        # Write the data to the CSV file
        writer.writerow([asin, title, price])
        # print(f"Data written for ASIN {asin}: Title - {title}, Price - {price}\n")

# Convert the failed_asins_list to a DataFrame
failed_asins_df = pd.DataFrame(failed_asins_list)

# Save the failed ASINs to a separate CSV file
failed_asins_df.to_csv('failed_asins-final2.csv', index=False)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\HikeOn - Sudhanva\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\

...

Title: Slide Tek 17.7 x 12.2 x 3.2 Inch Catering Trays, 10 Grease-Resistant Catering Boxes - Cover With Window Included, Side Lock, Kraft Paper Catering Food Containers, Recyclable, Inserts Sold Separately
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $2499
No relevant price section found.
Processing ASIN: B01CZD8OFC
Title: Restaurantware-39 x 20 x 8 Inch Anti-Fatigue Mat, 1 Extra-Padded Kitchen Mat - Beveled Safety Edges, Non-Slip, Black Plastic Kitchen Sink Rug, Shock-Resistant, For Home, Office, Or Laundry Room
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $113.01
Price found: $113
                 
                  ..01
Processing ASIN: B0DHLGRSBT
Title: Restaurantware 9018/Take Take Out Boxes, 100 count, Black
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fall

Title: Restaurantware 9143/Food Food Buckets, 5 lb/10, Blue
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $12.58
Price found: $12
                 
                  ..58
Processing ASIN: B017OY2A4S
Title: Restaurantware Candy Box PET Transparent Boxes Clear Gift Boxes for Wedding Party and Baby Shower Favors - Tuck Top with Sleeve - 1.4"L x 1.5"W x 2.8"H - 100ct
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $60.20
Price found: $60
                 
                  ..20
Processing ASIN: B082BXJ93P
Title: Sweet Vision 10 Inch x 8.25 Inch Transparent Cake Boxes, 10 Red Lid Clear Cake Boxes - Grease Resistant Base, Red Ribbon, Clear Plastic Birthday Cake Boxes, Bird Accent, For Weddings Or Birthdays
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: 

Title: Fiore 4 x 11 Inch Artificial Crape Myrtles, 12 Potted Faux Flowers - 1 Bloom, Use As Centerpieces, Home Decor, Or Office Decor, Purple Plastic Faux Plants In Plastic Pots, For Weddings Or Events,
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $267.87
Price found: $267
                 
                  ..87
Processing ASIN: B082XX7D8W
Title: Nature Tek 11.75 x 7.5 Inch Cutting Board With Handle, 1 Square Paddle Cutting Board - Shatterproof, Lightweight, Natural Wood Vintage Cheese Board, For Chopping And Serving Bread, Meat Or Charcuterie
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $2499
No relevant price section found.
Processing ASIN: B01CZD90A0
Title: Sapone del Fiore 4 x 8.7 Inch Soap Flower Petals In Plastic Pots, 12 Scented Rose Shaped Soaps - 7 Blooms, Gift for Anniversary, Mother's Day, Or Valentine’s Day, R

Title: 13.2 x 13.2 x 2 Inch Macaron Containers, 100 Shockproof Macaron Boxes - Fits 48 Macarons, With Clear Lids, Black Plastic Macaron Packaging, Use At Bakeries or Parties, For Displaying or Gifting
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $304.42
Price found: $304
                 
                  ..42
Processing ASIN: B082XX35V6
Title: Nature Tek 15 x 14 x 1.5 Inch Rustic Cake Stand, 1 Durable Footed Serving Tray - 3-Legged, Slab Design, Natural Wood Rustic Food Display Stand, Shatterproof, For Hors d'oeuvres, Appetizers, And Snacks
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $2499
No relevant price section found.
Processing ASIN: B01CZD9000
Title: Fiore 4 x 14 Inch Artificial Orchids, 12 Potted Fake Orchids - 5 Blooms, Use As Table Centerpieces, Home Decor, Or Office Decor, White Plastic Faux Orchids, In Plasti

Title: Restaurantware 8090/Stovetop Stovetop Griddles, 10 Count, Black
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $88.09
Price found: $88
                 
                  ..09
Processing ASIN: B084Q9W337
Title: Clean Tek Professional 15.5 x 6 Inch Disposable Shoe Covers, 100 No-Slip Shoe Guards - Durable, Protects Floors And Carpets, One Size Fits Most, Black Polypropylene Disposable Booties, For Painters Or Realtors
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: $14.91
Price found: $14
                 
                  ..91
Processing ASIN: B06XQ8RMWV
Title: Met Lux 2.5 Inch Deep Food Pans, 1 1/9 Size Commercial Food Storage Container - Freezer-Safe, Break-Resistant, Clear Plastic Cold Pans, Dishwasher-Safe, For Kitchens, Restaurants, Or Cafeterias
Attempting to find price in multiple sections.
Attempting to find pri

In [3]:
price_df= pd.read_csv('amazon_pricing1.csv')
price_df

,ASIN,Title,Price
0,B07GWFFTZC,Life Protector Gray Small Cut-Resistant Glove,$18.23
1,B01N7STJLG,Restaurantware-Sensei 6 Inch Japanese Boning K...,NaN
2,B072PTGTX7,"16 Ounce Disposable Coffee Cups, 500 Spiral Wa...",$145\n \n ..40
3,B07GWCF5G2,Life Protector Gray Small Cut-Resistant Glove,$16.55
4,B06XYK419C,"Met Lux 9 Ounce Small Casserole Dish With Lid,...",$20\n \n ..09
...,...,...,...
211,B082XX46RG,Nature Tek 11.5 x 11.1 x 1 Inch Rustic Cake St...,$2499
212,B07GVZDBSX,"Sweet Vision 2 x 1 Inch Wedding Favor Boxes, 1...",$63\n \n ..76
213,B082BX96FR,Sweet Vision 8.5 Inch x 6.75 Inch Transparent ...,$2499
214,B01CZD94BU,Sapone del Fiore 4 x 6 Inch Soap Flower Petals...,$267\n \n ..87


In [4]:
import pandas as pd

# Assuming review_check_df and review_check_df1 are already defined
price_df = pd.concat([price_df], ignore_index=True)

# Remove duplicate rows based on the 'ASIN' column
price_df = price_df.drop_duplicates(subset='ASIN')

# Check the number of rows to ensure correctness
print(f"Number of rows after removing duplicates based on 'ASIN': {price_df.shape[0]}")

price_df

Number of rows after removing duplicates based on 'ASIN': 216


,ASIN,Title,Price
0,B07GWFFTZC,Life Protector Gray Small Cut-Resistant Glove,$18.23
1,B01N7STJLG,Restaurantware-Sensei 6 Inch Japanese Boning K...,NaN
2,B072PTGTX7,"16 Ounce Disposable Coffee Cups, 500 Spiral Wa...",$145\n \n ..40
3,B07GWCF5G2,Life Protector Gray Small Cut-Resistant Glove,$16.55
4,B06XYK419C,"Met Lux 9 Ounce Small Casserole Dish With Lid,...",$20\n \n ..09
...,...,...,...
211,B082XX46RG,Nature Tek 11.5 x 11.1 x 1 Inch Rustic Cake St...,$2499
212,B07GVZDBSX,"Sweet Vision 2 x 1 Inch Wedding Favor Boxes, 1...",$63\n \n ..76
213,B082BX96FR,Sweet Vision 8.5 Inch x 6.75 Inch Transparent ...,$2499
214,B01CZD94BU,Sapone del Fiore 4 x 6 Inch Soap Flower Petals...,$267\n \n ..87


In [5]:
import pandas as pd

# Assuming your DataFrame is named `df` and the price column is named 'Price'

# Remove newline characters and excessive spaces
price_df['Price'] = price_df['Price'].str.replace(r'\n', '', regex=True)  # Remove newline characters
price_df['Price'] = price_df['Price'].str.replace(r'\s+', '', regex=True)  # Remove any excessive whitespace

# If the price is in a format like "$174..53", we need to handle this
# Replace '..' with a single '.' to correct the format
price_df['Price'] = price_df['Price'].str.replace(r'\.\.', '.', regex=True)
price_df = price_df[['ASIN','Title', 'Price']]
# Verify the changes
print(price_df[['ASIN','Title', 'Price']])

           ASIN                                              Title    Price
0    B07GWFFTZC      Life Protector Gray Small Cut-Resistant Glove   $18.23
1    B01N7STJLG  Restaurantware-Sensei 6 Inch Japanese Boning K...      NaN
2    B072PTGTX7  16 Ounce Disposable Coffee Cups, 500 Spiral Wa...  $145.40
3    B07GWCF5G2      Life Protector Gray Small Cut-Resistant Glove   $16.55
4    B06XYK419C  Met Lux 9 Ounce Small Casserole Dish With Lid,...   $20.09
..          ...                                                ...      ...
211  B082XX46RG  Nature Tek 11.5 x 11.1 x 1 Inch Rustic Cake St...    $2499
212  B07GVZDBSX  Sweet Vision 2 x 1 Inch Wedding Favor Boxes, 1...   $63.76
213  B082BX96FR  Sweet Vision 8.5 Inch x 6.75 Inch Transparent ...    $2499
214  B01CZD94BU  Sapone del Fiore 4 x 6 Inch Soap Flower Petals...  $267.87
215  B06XYPGLNP  Met Lux 13.5 x 10.5 x 10.6 Inch Cutting Board ...   $57.52

[216 rows x 3 columns]


In [6]:
price_df.to_csv("price_file_final.csv",index=False)

In [3]:
import pandas as pd
import datetime
# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
price_df1= pd.read_excel(r"C:\Users\sudha\Amazon validation script\webscrape_catalog as on 2024-12-23.xlsx")
price_df1

,ASIN,Age Range Description,Base Type,Best Sellers Rank,Bottle Type,Bottom Diameter,Bowl,Bowl Capacity,Bowl Material,Brand,...,bullet_point7,bullet_point8,bullet_point9,buybox_owner,image_block,product_description,product_information,product_specifications,products_overview,product_type
0,B00DOOFTAU,NaN,NaN,"#300,008 in Health & Household (",NaN,NaN,NaN,NaN,Plastic,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Infuse a touch of elegance into your serving s...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",DISHWARE_BOWL
1,B00NJSYCMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Make it easy for guests to sip and snack with ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",DISHWARE_PLATE
2,B00DOOCR7I,NaN,NaN,"#510,239 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",A rustic and unique alternative to serving sna...,Product information\n \n\n\n\n\n\n\n...,Product Material: Bamboo ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",FOOD_WRAP
3,B007Y1LJIU,NaN,NaN,"#464,324 in Kitchen & Dining (",NaN,NaN,NaN,NaN,Plastic,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Impress your guests with our 2-ounce Square Se...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",DISHWARE_BOWL
4,B00DOOD7W2,NaN,NaN,"#710,902 in Kitchen & Dining (",NaN,NaN,NaN,NaN,Wood,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Serve small portions or samples in style with ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",DISHWARE_BOWL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11990,B0DD8FDDFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Immerse your guests in a charming dessert appr...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DD8F...",NaN
11991,B0DD8PHHHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restaurantware,...,NaN,NaN,NaN,NaN,"P.when('A').register(""ImageBlockATF"", function...",Upgrade your kitchen with our Bundle: Met Lux ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DD8P...",SAUTE_FRY_PAN
11992,B0D9M3P8KC,NaN,NaN,"#509,468 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,Restaurantware,...,NaN,NaN,NaN,"<div class=""a-section a-spacing-base offer-dis...","P.when('A').register(""ImageBlockATF"", function...",Whip up tasty meals with our Met Lux Aluminum ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0D9M3...",GRIDDLE_PLATE
11993,B0CZB2BHSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"P.when('A').register(""ImageBlockATF"", function...",NaN,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CZB2...",NaN


In [9]:
import pandas as pd

# Assuming review_check_df and review_check_df1 are already defined
price_df = pd.concat([price_df1], ignore_index=True)

# Remove duplicate rows based on the 'ASIN' column
price_df = price_df.drop_duplicates(subset='ASIN')

# Check the number of rows to ensure correctness
print(f"Number of rows after removing duplicates based on 'ASIN': {price_df.shape[0]}")

price_df

Number of rows after removing duplicates based on 'ASIN': 11624


,ASIN,Age Range Description,Base Type,Best Sellers Rank,Bottle Type,Bottom Diameter,Bowl,Bowl Capacity,Bowl Material,Brand,...,bullet_point5,bullet_point6,bullet_point7,bullet_point8,bullet_point9,product_description,product_information,product_specifications,products_overview,product_type
0,B0BYZNCF4L,NaN,NaN,"#16,232 in Industrial & Scientific (",NaN,NaN,NaN,NaN,NaN,NaN,...,50 CORRUGATED FLATBREAD BOXES: Measuring 14 in...,NaN,NaN,NaN,NaN,Pack your Roman-style pizzas by using our Eco ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0BYZN...",FOOD_STORAGE_CONTAINER
1,B0CN71H39G,NaN,NaN,"#980,952 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,Restaurantware,...,500 BAKING CUPS PER ORDER: Measuring 2.8 inche...,NaN,NaN,NaN,NaN,Give your dessert presentation a unique touch ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CN71...",BAKING_CUP
2,B0DG2X81R5,NaN,NaN,"#50,290 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,Restaurantware,...,6 FRYING PANS: Measuring 16.7 inches in diamet...,NaN,NaN,NaN,NaN,Create culinary masterpieces with our Met Lux ...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DG2X...",SAUTE_FRY_PAN
3,B0CGXKC2SP,NaN,NaN,"#403,802 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,NaN,...,100 CONTAINERS PER ORDER: These salad to-go bo...,NaN,NaN,NaN,NaN,Protect patrons' take out meals with our Tampe...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CGXK...",FOOD_STORAGE_CONTAINER
4,B0D3F8KD9Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restaurantware,...,20 TABLE COVERS: These roundtable covers measu...,NaN,NaN,NaN,NaN,Dress up your dining tables with our Table Tek...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0D3F8...",TABLECLOTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11900,B0DDQBQTG1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restaurantware,...,100 CONTAINERS PER ORDER: These take out boxes...,NaN,NaN,NaN,NaN,Serve maki sushi with our Matsuri Vision 17-ou...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DDQB...",DISHWARE_PLATE
11901,B0CSG46JFH,NaN,NaN,"#177,147 in Industrial & Scientific (",NaN,NaN,NaN,NaN,NaN,NaN,...,400 FOOD BOATS PER ORDER: These food trays mea...,NaN,NaN,NaN,NaN,Take your catering to new heights using our Bi...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CSG4...",FOOD_STORAGE_CONTAINER
11902,B017OY23GS,NaN,NaN,"#507,117 in Kitchen & Dining (",NaN,NaN,NaN,NaN,NaN,NaN,...,SOLD IN BULK: These takeout lunch boxes are sh...,NaN,NaN,NaN,NaN,"Ideal for sushi rolls, sandwiches, wraps, chip...",Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B017OY...",FOOD_STORAGE_CONTAINER
11903,B0CSYYM1JJ,NaN,NaN,"#67,478 in Health & Household (",NaN,NaN,NaN,NaN,NaN,NaN,...,25 BOXES PER ORDER: Measuring 4 inches long by...,NaN,NaN,NaN,NaN,Carry your gifts easily using our Bio Tek Fren...,Product information\n \n\n\n\n\n\n\n...,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CSYY...",GIFT_WRAP


In [10]:
price_df.columns.all



<bound method Index.all of Index(['ASIN', 'Age Range Description', 'Base Type', 'Best Sellers Rank',
       'Bottle Type', 'Bottom Diameter', 'Bowl', 'Bowl Capacity',
       'Bowl Material', 'Brand',
       ...
       'bullet_point5', 'bullet_point6', 'bullet_point7', 'bullet_point8',
       'bullet_point9', 'product_description', 'product_information',
       'product_specifications', 'products_overview', 'product_type'],
      dtype='object', length=146)>

In [11]:
import pandas as pd

# Assuming your DataFrame is named `df` and the price column is named 'Price'

# Remove newline characters and excessive spaces
price_df['Price'] = price_df['Price'].str.replace(r'\n', '', regex=True)  # Remove newline characters
price_df['Price'] = price_df['Price'].str.replace(r'\s+', '', regex=True)  # Remove any excessive whitespace

# If the price is in a format like "$174..53", we need to handle this
# Replace '..' with a single '.' to correct the format
price_df['Price'] = price_df['Price'].str.replace(r'\.\.', '.', regex=True)
price_df = price_df[['ASIN','Title', 'Price']]
# Verify the changes
print(price_df[['ASIN','Title', 'Price']])


             ASIN                                              Title    Price
0      B0BYZNCF4L  Restaurantware Eco Pie 14 x 7 x 1.5 Inch Flatb...   $64.31
1      B0CN71H39G  Restaurantware Panificio 4.5 Ounce Baking Cups...  $141.66
2      B0DG2X81R5  Restaurantware Met Lux 9.5 Inch Fry Pans, 6 Li...  $106.92
3      B0CGXKC2SP  Tamper Tek 48 Ounce Rectangle Take Out Contain...   $83.24
4      B0D3F8KD9Y  Restaurantware - Table Tek 84 Inch Gingham Tab...   $49.12
...           ...                                                ...      ...
11900  B0DDQBQTG1  Restaurantware-Matsuri Vision 5.1 x 5.1 x 1.9 ...   $26.24
11901  B0CSG46JFH  Restaurantware Bio Tek 4 Ounce Paper Boats 400...   $47.92
11902  B017OY23GS  Restaurantware Cafe Vision 41 Ounces Food Boxe...  $117.24
11903  B0CSYYM1JJ  Bio Tek 4 x 2.5 x 2.5 Inch Gable Boxes For Par...    $9.02
11904  B0D3F6JQ5J  Restaurantware Bio Tek 6 Ounce Boat Paper Boat...    $8.31

[11624 rows x 3 columns]


In [12]:
price_df.to_csv("price_file_final4.csv",index=False)

In [13]:
price_df.Price[1]

'$141.66'

In [ ]:
soup2